# Interpolation of a biderectional curve network

In [ ]:
from pygbs import gbs
import numpy as np
from math import radians, cos

In [ ]:
crv_2d = gbs.bscurve2d_approx_from_points("../../../tests/in/e1098.dat",3,gbs.KnotsCalcMode.CHORD_LENGTH,1)


nv = 5
r1 = 0.3
r2 = 1.
r  = np.linspace( r1, r2, nv)
g  = np.linspace(radians(15), radians(50), nv)
cax= 0.4
l  = np.linspace( 0, 0.1 * cax, nv)
c  = np.array(list(map(lambda g_ : cax * cos(g_), g)))

def make_3d_foil(params):
    r_, g_, c_, l_ = params
    crv_2d_ = gbs.BSCurve2d(crv_2d)
    gbs.translate(crv_2d_,[-0.5,0.])
    gbs.rotate(crv_2d_, g_)
    gbs.scale(crv_2d_, c_)
    gbs.translate(crv_2d_,[0.,l_])
    gbs.translate(crv_2d_,[0.5,0.])
    return gbs.to_bscurve_3d(crv_2d_, r_)
    
    
foils = list(map(make_3d_foil, zip(r, g, c, l)))


In [ ]:
import pyvista as pv
from pygbs import vistaplot as gbv

# Initialize the PyVista plotter
plotter = pv.Plotter()

# Add the defined curves to the plotter for visualization
gbv.add_curves_to_plotter(foils, plotter)

# Display the plot with the curves
plotter.show()

In [ ]:
Lu = gbs.loft(foils)

In [ ]:
plotter = pv.Plotter()
gbv.add_curves_to_plotter(foils, plotter)
gbv.add_surfaces_to_plotter([Lu], plotter, nu = 300, )

plotter.show()

In [ ]:
u1, u2 = crv_2d.bounds()
nu = 7
u = np.linspace(u1, u2,nu)
v = np.linspace(0., r2-r1, nv)

q = 3

def make_span_curve(u_):
    pts = [ foil(u_) for foil in foils]
    begin =( v[0], pts[0] )
    end =( v[-1], pts[-1] )
    contraints = [(v_, pt, 0) for v_, pt in zip(v[1:-1], pts[1:-1])]
    contraints.append( (0., [0,0,1], 1) )
    return gbs.interpolate(begin, end, contraints, q)

spans = list(map(make_span_curve, u))

plotter = pv.Plotter()
gbv.add_curves_to_plotter(foils, plotter)
gbv.add_surfaces_to_plotter([Lu], plotter, nu = 300, )
gbv.add_curves_to_plotter(spans, plotter)

plotter.show()

In [ ]:
Lv = gbs.loft(spans)

plotter = pv.Plotter()
gbv.add_curves_to_plotter(foils, plotter)
gbv.add_surfaces_to_plotter([Lv], plotter, nu = 300, )
gbv.add_curves_to_plotter(spans, plotter)

plotter.show()

In [ ]:
pts = [ [ foil(u_) for foil in foils] for u_ in u ]

T = gbs.interpolate(pts)